# App description

This app prepares for an upcoming match-up between two fantasy teams. This is based on a league with *Head to Head Each Category* scoring type and 8 stat categories (FG%, FT%, 3PM, REB, AST, STL, BLK, TO, PTS).

* It compares their schedule (number of starter players and unused players) 
* It compares the teams' historic stats up to this round
* Simulates/projects the match-up based on the players' stats and schedule. 
* Allows for scenarios, such as replace player X with player Y, etc. (feature not currently available, will be added soon)


Further details on [this Medium blog post](https://g-giasemidis.medium.com/nba-fantasy-analytics-with-python-on-epsn-f03f10a60187).

In [1]:
import sys
import pandas as pd
sys.path.append('../')
from src.EspnFantasyMatchUp import EspnFantasyMatchUp
from src.utils.io_json import read_json

In [2]:
config = read_json("../config/config.json")

STAT_PERIOD_DICT = {
    'season average': '002022',
    'last 7 days average': '012022',
    'last 15 days average': '022022',
    'last 30 days average': '032022',
    "season's projections": '102022',
    'previous season average': '002021'
}

In [3]:
cookies = config['cookies']
league_settings = config['league']

In [9]:
week = 19

start_date = "2022-03-20"
end_date = "2022-03-20"

home_team = "DGGG"
away_team = "MAR"

use_current_score = True
stat_type_code = "season average"

In [10]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [11]:
espn = EspnFantasyMatchUp(
    cookies, league_settings,
    week, 
    home_team=home_team, away_team=away_team, 
    start_date=start_date, end_date=end_date,
    stat_type_code=STAT_PERIOD_DICT[stat_type_code]
)

2022-03-20 11:01:35,433: EspnFantasyLeague:set_league_team_division_settings INFO: 18 teams participating
2022-03-20 11:01:35,435: EspnFantasyLeague:set_league_team_division_settings INFO: 8 starters per team


# Head to head comparison of season historic stats

In [12]:
espn.h2h_season_stats_comparison().astype("O")

2022-03-20 11:01:36,314: EspnFantasyMatchUp:h2h_season_stats_comparison INFO: projected score DGGG-MAR: 5-4-0


teamAbbr,DGGG,MAR,Projected Winner
FG%,0.463,0.4617,DGGG
FT%,0.7772,0.7621,DGGG
3PM,1379,1158,DGGG
REB,3869,4230,MAR
AST,2468,2161,DGGG
STL,646,675,MAR
BLK,402,459,MAR
TO,1229,1074,MAR
PTS,10298,9722,DGGG


# Compare Schedule (players per day)

In [13]:
espn.compare_schedules().astype("O")

2022-03-20 11:01:36,719: utils:extract_player_stat WARNING: Player Ben Simmons does not have requested data
2022-03-20 11:01:36,721: utils:extract_player_stat WARNING: Player Jonathan Isaac does not have requested data


,Count-DGGG,Count_valid-DGGG,Unused_subs-DGGG,Count-MAR,Count_valid-MAR,Unused_subs-MAR
2022-03-20,5,5,0,7,7,0
Total,NaN,5,NaN,NaN,7,NaN


# Simulate Schedule

In [14]:
espn.simulation(use_current_score=use_current_score)

2022-03-20 11:01:38,676: EspnFantasyMatchUp:simulation INFO: Player stats type season average


,HomeWinProb,DGGG,MAR
FG%,0.002,0.474968,0.503629
FT%,99.998,0.764846,0.708328
3PM,100.000,102.438240,65.066970
REB,0.000,414.658240,480.490460
AST,100.000,269.310170,214.201100
STL,0.000,52.079430,74.981370
BLK,7.591,44.167650,47.867220
TO,1.747,126.637300,117.392480
PTS,96.428,998.391490,975.939210


# Scenarios

Add or remove players from the roster of either the home or the away team

```
home_scn_players={
    "add": {
        <plyaer_name>: [<date1>, <date2>],
        <player_name2>: [<date1>]
    }, 
    "remove": {
        <plyaer_name3>: [<date1>, <date2>],
        <player_name4>: [<date1>]
    }, 
}
```
Similar of the away players, using the `away_scn_players` argument. 

Leave date list empyt if you want to include all dates in the schedule between `start_date` and `end_date`

In [21]:
scn = EspnFantasyMatchUp(
    cookies, league_settings,
    week, home_team=home_team, away_team=away_team, start_date=start_date, end_date=end_date,
    home_scn_players={
        "add": {"Juancho Hernangomez": []}, 
        "remove": {}
    }, 
    stat_type_code="032022"
)

2022-03-20 11:16:11,063: EspnFantasyLeague:set_league_team_division_settings INFO: 18 teams participating
2022-03-20 11:16:11,070: EspnFantasyLeague:set_league_team_division_settings INFO: 8 starters per team


In [22]:
scn.compare_schedules().astype("O")

2022-03-20 11:16:13,156: utils:extract_player_stat WARNING: Player Lonzo Ball does not have requested data
2022-03-20 11:16:14,460: utils:extract_player_stat WARNING: Player Ben Simmons does not have requested data
2022-03-20 11:16:14,462: utils:extract_player_stat WARNING: Player Jonathan Isaac does not have requested data


,Count-DGGG,Count_valid-DGGG,Unused_subs-DGGG,Count-MAR,Count_valid-MAR,Unused_subs-MAR
2022-03-20,6,6,0,7,7,0
Total,NaN,6,NaN,NaN,7,NaN


In [23]:
scn.simulation(use_current_score=True)

2022-03-20 11:16:14,498: EspnFantasyMatchUp:simulation INFO: Player stats type last 30 days average


,HomeWinProb,DGGG,MAR
FG%,0.012,0.475136,0.502926
FT%,99.994,0.762729,0.708887
3PM,100.000,102.799560,63.537880
REB,0.000,423.970490,480.602170
AST,100.000,276.712980,212.431880
STL,0.000,53.621250,74.494720
BLK,30.698,46.714500,47.905810
TO,0.332,128.397960,116.481360
PTS,99.972,1014.460190,970.866910
